In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Define constants
IMAGE_SIZE = 150
BATCH_SIZE = 32

# Set up data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Splitting data into training and validation sets
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [3]:
train_generator = train_datagen.flow_from_directory(
    'Chest/Train',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    classes=['Normal', 'PNEUMONIA']
)

test_generator = test_datagen.flow_from_directory(
    'Chest/Test',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    classes=['Normal', 'PNEUMONIA']  # Explicitly specify class labels
)

Found 5216 images belonging to 2 classes.
Found 696 images belonging to 2 classes.


In [4]:

# Create the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    BatchNormalization(),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    BatchNormalization(),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    BatchNormalization(),

    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    BatchNormalization(),

    Flatten(),

    Dense(512, activation='relu'),
    Dropout(0.5),
    BatchNormalization(),

    Dense(256, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),

    Dense(1, activation='sigmoid')
], name='PNEUMONIA_classifier')

In [5]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [6]:
# Train the model using the generators
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=10,
    validation_data=test_generator,
    validation_steps=test_generator.samples // BATCH_SIZE
)

Epoch 1/10
163/163 [==============================] - 261s 2s/step - loss: 0.3091 - accuracy: 0.8882 - val_loss: 2.3901 - val_accuracy: 0.6161
Epoch 2/10
163/163 [==============================] - 211s 1s/step - loss: 0.2294 - accuracy: 0.9151 - val_loss: 3.0889 - val_accuracy: 0.6161
Epoch 3/10
163/163 [==============================] - 190s 1s/step - loss: 0.2081 - accuracy: 0.9191 - val_loss: 2.2704 - val_accuracy: 0.6131
Epoch 4/10
163/163 [==============================] - 187s 1s/step - loss: 0.1858 - accuracy: 0.9289 - val_loss: 0.5053 - val_accuracy: 0.7649
Epoch 5/10
163/163 [==============================] - 188s 1s/step - loss: 0.1794 - accuracy: 0.9293 - val_loss: 0.5270 - val_accuracy: 0.8095
Epoch 6/10
163/163 [==============================] - 185s 1s/step - loss: 0.1606 - accuracy: 0.9404 - val_loss: 1.3661 - val_accuracy: 0.6518
Epoch 7/10
163/163 [==============================] - 185s 1s/step - loss: 0.1457 - accuracy: 0.9450 - val_loss: 1.5239 - val_accuracy: 0.5521

In [7]:
# Save the model
model.save('PNEUMONIA_classifier_model.h5')

C:\Users\brigh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [12]:
# VGG 16 model
from tensorflow.keras.applications import VGG16

# Load the VGG16 model
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

# Freeze all the layers
for layer in vgg16_model.layers:
    layer.trainable = False

# Create a new model on top of the pre-trained base model
model2 = Sequential([
    vgg16_model,
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [13]:
# Compile the model
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
# Train the model using the generators
model2.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=10,
    validation_data=test_generator,
    validation_steps=test_generator.samples // BATCH_SIZE
)

Epoch 1/10
1/1 [==============================] - 8s 8s/step - loss: 0.9252 - accuracy: 0.3750 - val_loss: 1.5498 - val_accuracy: 0.5000
Epoch 2/10
1/1 [==============================] - 6s 6s/step - loss: 0.6847 - accuracy: 0.7500 - val_loss: 0.8771 - val_accuracy: 0.6094
Epoch 3/10
1/1 [==============================] - 6s 6s/step - loss: 0.3082 - accuracy: 0.8125 - val_loss: 0.1977 - val_accuracy: 0.9062
Epoch 4/10
1/1 [==============================] - 6s 6s/step - loss: 0.0315 - accuracy: 1.0000 - val_loss: 0.1021 - val_accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 6s 6s/step - loss: 0.1018 - accuracy: 1.0000 - val_loss: 0.1086 - val_accuracy: 0.9531
Epoch 6/10
1/1 [==============================] - 7s 7s/step - loss: 0.0737 - accuracy: 1.0000 - val_loss: 0.0988 - val_accuracy: 0.9531
Epoch 7/10
1/1 [==============================] - 6s 6s/step - loss: 0.0306 - accuracy: 1.0000 - val_loss: 0.0715 - val_accuracy: 0.9688
Epoch 8/10
1/1 [=========================

In [15]:
# Save the model
model2.save('tomato_classifier_VGG16.h5')

C:\Users\brigh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
# Assuming you've trained and saved your model already...

# Load the saved models
model = tf.keras.models.load_model('PNEUMONIA_classifier_model.h5')
# loaded_model = tf.keras.models.load_model('tomato_classifier_VGG16.h5')

# Now, assuming you have a new image you want to classify
import numpy as np
from tensorflow.keras.preprocessing import image

# Load and preprocess the new image
new_image_path = 'Chest/Test/PNEUMONIA/0111.jpeg'  # Replace with the path to your new image
new_img = image.load_img(new_image_path, target_size=(IMAGE_SIZE, IMAGE_SIZE))
new_img_array = image.img_to_array(new_img)
new_img_array = np.expand_dims(new_img_array, axis=0)
new_img_array /= 255.  # Normalize the image

# Make a prediction using the loaded model
prediction = model.predict(new_img_array)
# prediction1 = loaded_model.predict(new_img_array)

# Interpret the prediction
print("prediction of model 1")
if prediction[0] < 0.5:
    print("The tomato is classified as 'Normal'.")
else:
    print("The tomato is classified as 'PNEUMONIA'.")

# print("prediction of model vgg16")
# if prediction1[0] < 0.5:
#     print("The tomato is classified as 'good'.")
# else:
#     print("The tomato is classified as 'bad'.")

1/1 [==============================] - 1s 625ms/step
prediction of model 1
The tomato is classified as 'PNEUMONIA'.


In [9]:
import pandas as pd
import os
import csv
import base64
import numpy as np
from PIL import Image

# Define constants
IMAGE_SIZE = 150  # Adjust this according to your image size
dataset_path = 'Chest'  # Replace this with your dataset path
csv_file_path = 'chest_dataset_with_images.csv'
excel_file_path = 'chest_dataset.xlsx'

# Open CSV file in write mode
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Filename', 'Label', 'Image', 'Set'])  # Write header

    # Iterate through Train and Test folders
    for folder in ['Train', 'Test']:
        for class_name in ['Normal', 'PNEUMONIA']:
            class_dir = os.path.join(dataset_path, folder, class_name)
            for filename in os.listdir(class_dir):
                file_path = os.path.join(class_dir, filename)

                # Check if the file is an image by attempting to open it with PIL
                try:
                    img = Image.open(file_path)
                    img = img.resize((IMAGE_SIZE, IMAGE_SIZE))
                except (IOError, OSError):
                    continue  # Skip if it's not an image file

                # Convert image to numpy array
                img_array = np.array(img)
                
                # Convert image array to base64 string
                img_str = img_array.tobytes()
                img_base64 = base64.b64encode(img_str).decode('utf-8')

                # Determine if the image is in the 'Train' or 'Test' set
                image_set = folder

                writer.writerow([file_path, class_name, img_base64, image_set])

# Read the CSV file into a pandas DataFrame
data = pd.read_csv(csv_file_path)

# Save the DataFrame to an Excel file
data.to_excel(excel_file_path, index=False)
